In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
#from xgboost import XGBClassifier
from sklearn.decomposition import PCA
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import AdaBoostClassifier
#import lightgbm as lgb
import joblib

In [ ]:
input_image = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
output_image = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [ ]:
input_image.shape

In [ ]:
output_image.shape

In [2]:
## 降维操作
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array

In [ ]:
pca_return_array = pca_hpi(input_image,30)  # 降维后Cr成为30

In [ ]:
pca_return_array.shape

In [ ]:
pca_return_array[0].shape


## patch的生成   3D tensor

In [9]:
# 可以提取出高光谱数据的一个channel的数据 144*144
def split_one_channel(A ,channel):
    d = [0]*145
    for i in range(145):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(145)))
    return e  
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


def recreate_whole_data(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    return whole_mirror_matrix_array


## 分割patch  一个维度的patch
def create_patch_one(A,fil):
    whole_arr = list()
    row = A.shape[0]-fil+1
    column = A.shape[1]-fil+1
    for i in range(row):
        for j in range(column):
            temp_arr = list()
            for z in range(fil):
                p = A[i+z,j:j+fil]
                temp_arr.append(p)
            one_arr = np.vstack(temp_arr)
            whole_arr.append(one_arr)
    return np.array(whole_arr)
    
def create_patch_whole(A,fil,dim):
    temp = [create_patch_one(split_one_channel(A,i),fil) for i in range(1,dim+1)]
    return np.array(temp)


def create_patch_cube(patch,fil,dim):
    temp = list()
    nums = patch[0].shape[0] # the nums of every dim of windows  n - f + 1
    for j in range(nums):
        temp.append(np.vstack([patch[i][j] for i in range(dim)]).reshape(dim,fil,fil))
    patch_cubes = np.array(temp)
    return patch_cubes
    
    

### 测试

In [ ]:
## 将每个 filxfil (3x3)的patcha取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array,3,30)

In [ ]:
patch_dim.shape  # 30 个dim  every dim 20499个window (sample)    window(3x3)

In [ ]:
143*143

In [ ]:
patch_cubes = create_patch_cube(patch_dim,3, 30)

In [ ]:
patch_cubes.shape  # 20499 samples   one sampe is a 3D tensor (3x3x30)

In [ ]:
## 测试成功

## 开始联合标签

In [ ]:
patch_cubes[0].shape

In [5]:
DIM = 30
FIL = 3

In [6]:
inp = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
out = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [7]:
## 降维
pca_return_array_alpha = pca_hpi(inp,DIM) 

[6.84937994e-01 2.35313708e-01 1.49635476e-02 8.21541515e-03
 6.95011265e-03 5.17010703e-03 3.99681431e-03 3.62359687e-03
 3.07127396e-03 2.93211894e-03 2.67352983e-03 2.49230109e-03
 2.24688362e-03 1.89388746e-03 1.69433609e-03 1.56043454e-03
 1.53162478e-03 1.35012965e-03 1.00138526e-03 9.24873106e-04
 8.47874114e-04 7.64322238e-04 6.64576759e-04 6.45645968e-04
 6.16310190e-04 5.61125098e-04 5.43138098e-04 5.15486796e-04
 4.20444839e-04 3.62877692e-04]


In [10]:
# padd 并镜像
pca_return_array_beta = recreate_whole_data(DIM,FIL,pca_return_array_alpha)


In [11]:
pca_return_array_beta.shape

(30, 147, 147)

In [ ]:
# pca_return_array_beta.transpose(2,1,0).shape

In [ ]:
# pca_return_array_beta.shape

In [12]:
## 将pca_return beta 变为 147--3  x  147--2  x 30 形状
pca_return_array_beta = pca_return_array_beta.transpose(2,1,0)

In [13]:
pca_return_array_beta.shape

(147, 147, 30)

In [14]:
pca_return_array_beta = pca_return_array_beta.transpose(1,0,2)

In [15]:
pca_return_array_beta.shape

(147, 147, 30)

In [16]:
## ## 将每个 filxfil (3x3)的patch取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array_beta,3,30)

In [18]:
patch_dim.shape  # 30 个dim  every dim 20735个window (sample)    window(3x3)

(30, 20735, 3, 3)

In [19]:
patch_cubes = create_patch_cube(patch_dim,3, 30)

In [20]:
patch_cubes.shape   # 20735个 sample 每个sample是一个patch 30dim  3x3

(20735, 30, 3, 3)